In [1]:
import os

In [2]:
!pip install wget

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!git clone https://github.com/tensorflow/models pretained

fatal: destination path 'pretained' already exists and is not an empty directory.


In [4]:
!cd pretained/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py .

In [5]:
VERIFICATION_SCRIPT = os.path.join(os.getcwd(), "pretained", 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')

In [6]:
#Installing Tensorflow

!pip install absl-py tensorflow


Defaulting to user installation because normal site-packages is not writeable
     |▍                               | 6.0 MB 1.5 MB/s eta 0:05:30^C

ERROR: Operation cancelled by user


In [7]:
VERIFICATION_SCRIPT

'/home/alpha/Code/Project/MerchantPlus/machinelearning/pretained/research/object_detection/builders/model_builder_tf2_test.py'

In [8]:
import object_detection

In [9]:
!pip install protobuf matplotlib==3.2

Defaulting to user installation because normal site-packages is not writeable
  Using cached protobuf-4.21.4-cp37-abi3-manylinux2014_x86_64.whl (408 kB)
     |████████████████████████████████| 12.4 MB 757 kB/s eta 0:00:01


In [10]:


CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'



In [11]:
!wget {PRETRAINED_MODEL_URL}

--2022-08-06 01:10:12--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 2404:6800:4009:814::2010, 142.250.67.240
Connecting to download.tensorflow.org (download.tensorflow.org)|2404:6800:4009:814::2010|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46042990 (44M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_32 100%[===================>]  43.91M  5.12MB/s    in 14s     

2022-08-06 01:10:27 (3.14 MB/s) - ‘ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz’ saved [46042990/46042990]



In [12]:
labels = [{'name':'Cup', 'id':1}, {'name':'Smartphone', 'id':2}, {'name':'Book', 'id':3}, {'name':'Bottle', 'id':4}]

In [13]:
with open(LABEL_MAP_NAME, 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')


In [14]:
ARCHIVE_FILES = os.path.join(os.getcwd(), 'images', 'archive.tar.gz')

In [15]:
ARCHIVE_FILES

'/home/alpha/Code/Project/MerchantPlus/machinelearning/images/archive.tar.gz'

In [16]:


import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format



In [17]:
config = config_util.get_configs_from_pipeline_file('pipeline.config')

In [18]:
config

{'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 300
       width: 300
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         truncated_normal_initializer {
           mean: 0.0
           stddev: 0.029999999329447746
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.9700000286102295
         center: true
         scale: true
         epsilon: 0.0010000000474974513
         train: true
       }
     }
     override_base_feature_extractor_hyperparams: true
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matcher {
     argmax_matcher {
       matched_threshold: 0.5
       unmatched_

In [19]:


pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile("pipeline.config", "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  



In [20]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4

In [21]:
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')

NameError: name 'paths' is not defined

In [22]:
MODEL_PATH = "/home/alpha/Code/Project/MerchantPlus/machinelearning/mdel/ssd_mobilenet_v2_320x320_coco17_tpu-8"
PIPELINE_CONFIG = os.path.join(os.getcwd(), "pipeline.config")

In [23]:
command = f"python model_main.py --model_dir={MODEL_PATH} --pipeline_config_path={PIPELINE_CONFIG} --num_train_steps=2000"

In [24]:
command


'python model_main.py --model_dir=/home/alpha/Code/Project/MerchantPlus/machinelearning/mdel/ssd_mobilenet_v2_320x320_coco17_tpu-8 --pipeline_config_path=/home/alpha/Code/Project/MerchantPlus/machinelearning/pipeline.config --num_train_steps=2000'

In [1]:


import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util



In [2]:
configs = config_util.get_configs_from_pipeline_file("pipeline.config")

In [3]:
configs


{'model': ssd {
   num_classes: 5
   image_resizer {
     fixed_shape_resizer {
       height: 300
       width: 300
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         truncated_normal_initializer {
           mean: 0.0
           stddev: 0.029999999329447746
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.9700000286102295
         center: true
         scale: true
         epsilon: 0.0010000000474974513
         train: true
       }
     }
     override_base_feature_extractor_hyperparams: true
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matcher {
     argmax_matcher {
       matched_threshold: 0.5
       unmatched_t

In [4]:
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

2022-08-06 02:46:09.844588: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-06 02:46:09.886778: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-08-06 02:46:09.886797: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-08-06 02:46:09.887199: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

In [9]:
ckpt.restore(os.path.join(os.getcwd(), 'model', 'ckpt')).expect_partial()

NotFoundError: Error when restoring from checkpoint or SavedModel at /home/alpha/Code/Project/MerchantPlus/machinelearning/model/ckpt: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for /home/alpha/Code/Project/MerchantPlus/machinelearning/model/ckpt
Please double-check that the path is correct. You may be missing the checkpoint suffix (e.g. the '-1' in 'path/to/ckpt-1').

In [24]:


import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util



In [19]:


# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file("pipeline.config")
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join("model", 'ckpt-3')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections



In [20]:


import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline



In [21]:
category_index = label_map_util.create_category_index_from_labelmap("label_map.pbtxt")

In [22]:
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened(): 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.8,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break
        
cap.release()
cv2.destroyAllWindows()

INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0


error: OpenCV(4.6.0) /io/opencv/modules/highgui/src/window.cpp:1267: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [23]:
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.6.0) /io/opencv/modules/highgui/src/window.cpp:1261: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'
